In [1]:
import smtplib
import numpy as np
import pandas as pd

In [105]:
def send_email(user, pwd, recipient, subject, body):
    import smtplib

    gmail_user = user
    gmail_pwd = pwd
    FROM = user
    TO = recipient if type(recipient) is list else [recipient]
    SUBJECT = subject
    TEXT = body

    # Prepare actual message
    message = """\From: %s\nTo: %s\nSubject: %s\n\n%s
    """ % (FROM, ", ".join(TO), SUBJECT, TEXT)
    try:
        server = smtplib.SMTP("smtp.gmail.com", 587)
        server.ehlo()
        server.starttls()
        server.login(gmail_user, gmail_pwd)
        server.sendmail(FROM, TO, message)
        server.close()
        print 'successfully sent the mail'
    except:
        print "failed to send mail"
        
def gradeStatus(grade):
    grade = float(grade)*100.0
    if grade >= 90.0:
        return ', which means you are doing great! Keep up the good work!'
    elif grade < 90.0 and grade >= 80.0:
        return ', which means your are doing well.'
    elif grade < 80.0 and grade >= 70.0:
        return ', which means you are doing ok, but could improve a bit.'
    elif grade < 70.0:
        return ', which means you are having some trouble, and we should talk if you want.'
    else:
        return '.'

def informalgreet():
    i = np.random.random_integers(0, high=4)
    greet = ['I hope life is treating you well.', 'How are you doing?', 
             'I am glad that you are in my class.', 'How\'s it going?', 
             'I hope things are going well for you.']
    return greet[i]

def signOff():
    i = np.random.random_integers(0, high=5)
    signOffs = ['Best,','Thanks,', 'Take care,', 'Regards,', 'Thank You,', 'Sincerely,']
    return signOffs[i]

def findMissing(x, head):
    missing = []
    for i in range(len(x)):
        if x[i]== 0 or x[i]== None or x[i]=='' or x[i]=='Null' or np.isnan(x[i]):
            missing.append(head[i])
    if len(missing) == 0:
        return ' You are not missing any assignments! Keep up the good work.'
    elif len(missing) == 1:
        return ' You are only missing the %s assignment.'%(missing[0])
    elif len(missing) == 2:
        message = ' You are missing your ' + missing[0] + ' and ' + missing[1] + '.'
        return message
    elif len(missing) > 2:
        messageBeg = ' You are missing your '
        messageMid = ', '.join(map(str, missing[0:-1]))
        message = messageBeg + messageMid + ', and ' + missing[-1] + '.'
        return message
    else:
        return ''

In [ ]:
rootsource = '/media/p/Transcend/PROJECTS/UMAR/data/21_Nov_20-50_Grades-GEO-1010-009-F15.csv'

In [60]:
ledger = pd.read_csv(rootsource, na_values=['(read only)','NaN'])
ledgerlist = list(ledger.iloc[0,:])

In [73]:
ledgerindices = []
for i in range(len(ledgerlist)):
    if ledgerlist[i]>0:
        ledgerindices.append(i)
ledgerindices = ledgerindices[1:]

totalPoints = 0
for i in ledgerindices:
    totalPoints += ledger.iloc[0,i]
ledger['studPoints'] = ledger[ledgerindices].sum(axis=1)
ledger['grade'] = ledger['studPoints']/ledger['studPoints'][0]
ledger['firstName'] = ledger['Student'].apply(lambda x: x.split(' ')[0],1)
ledger['lastName'] = ledger['Student'].apply(lambda x: x.split(' ')[1],1)
headers = ledger.columns[ledgerindices]
headCols = [i.split(' (')[0] for i in list(headers)]
ledger['email'] = ledger['SIS Login ID'][i]

In [1]:
for i in range(len(ledger)):
    name = ledger['firstName'][i]
    studList = ledger.iloc[i,ledgerindices]
    studPoints = ledger['studPoints'][i]
    missing = findMissing(list(studList), headCols)
    grade = str(round(ledger['grade'][i]*100,1))+'%'
    gradestatus = gradeStatus(ledger['grade'][i])
    email = str(ledger['SIS Login ID'][i])+'@slcc.edu'
    print email
    personalMess = ''
    body = ('''%s,\n     %s You have %s out of %s points in my class (%s)%s%s%s\n\n%s\nPaul'''% (name, informalgreet(), studPoints, totalPoints, grade, gradestatus, missing, personalMess, signOff()))


NameError: name 'ledger' is not defined